In [1]:
from argparse import ArgumentParser, ArgumentDefaultsHelpFormatter

from addition_data import *
import sys
sys.path.insert(0,'../../')

from rcalgo.tf.tfmodels.seq2seq import *
from collections import Counter
from rcalgo.tf.utils.text_utils import *

import random
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split

In [2]:
DIGITS = 3
SAMPLE_SIZE = 50000
nwords = 0
maxlen = DIGITS*2 + 3
target_maxlen = DIGITS + 2 #add eos
x, y = generate_addtion_pairs(sample_size=SAMPLE_SIZE, digits=DIGITS, raw_text=True)
#x_seq = np.sum(x > 0, axis=1) #np.zeros(shape=(SAMPLE_SIZE)) + maxlen
#y_seq = np.sum(y > 0, axis=1) + 1

Generating data...
Total addition questions: 50000


In [3]:
word_freq = word_count(x, word_seg=False)
term_id_map = build_word2idx(word_freq, freq_threshold=1)
id_term_map = dict([(v, k) for k,v in term_id_map.items()])

In [4]:
from rcalgo.tf.utils.train_config import *

args = parse_initial_training_args(
    [
        '--learning-rate', '0.001',
        '--n-epochs', '20',
        '--opt', 'adam',
        '--batch-size', '128',
        '--p-coef', '0',
        '--keep-prob','0.5',
        '--embedding-size', '256',
        '--model-dir', 'model/',
        '--model-name', 'seq2seq',
        '--num-layers', '1',
        '--gpu-id', '0',
        '--gpu-num', '1',
        '--hidden-size','256',
        '--maxlen', str(maxlen),
        '--epoch-save',
        '--use-residual',
        '--position-type', '2,3,4,5',
        '--beam-width', '200',
        '--sample-decay-factor', '5000',
        '--oov-buckets', '2',
        '--expand-input',
        '--epoch-save',
       # '--sampling-probability', '0.01',
      #  '--epoch-save',
    ]
)

args.add_conv_flag = True
args.output_maxlen = target_maxlen
args.sampled_softmax = False
args.num_sampled = 1
args.sample_prob = 1.0
args.beam_width = 0
args.attention_size = 0

In [5]:
#tf.nn.rnn_cell.LSTMStateTuple(seq2seq_model.rnn_output_list[0])

In [7]:
graph_to_use = tf.Graph()
config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth=True
seq2seq_model = Seq2SeqModel(args, graph_to_use, word_dict=term_id_map)
#seq2seq_model.build_model()
#seq2seq_model.build_model_summary()
#df = seq2seq_model.model_summary()
#display(df)
#seq2seq_model.run([x,np.zeros([len(x), 2, 256]),y], test_size=0.2)


In [ ]:
with seq2seq_model.graph.as_default():
    input_str = tf.get_collection('default_inputs')[0]
    target_str = tf.get_collection('default_targets')[0]
    output = tf.get_collection('default_outputs')[1]

In [ ]:
outputs = seq2seq_model.job.session.run(output, 
                                        feed_dict={input_str:x[0:10]})

In [ ]:
def out_print(num_arr, id_term_map, eos=17):
    text = ''
    for i in num_arr:
        if i == eos:
            break
        else:
            text += id_term_map[i]
    return text

In [ ]:
for i in range(0, 10):
    print(x[i], y[i], out_print(outputs[:,:,0][i], id_term_map))